# Gaussian dictionnary learning demo

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

from gaussian_dictionary_learning import DictionaryLearningAlgorithm
from utils import Gaussians, MetricsUtils

from sklearn.decomposition import PCA

# load the demo data, this is represents one grain in a sample
X = np.load("Data/data.npy")

# Load the real spectra and activation map
phases = np.array([np.load("Data/aspim020_3ph_nps_gauss_noisemap_p" + str(i) + ".npy") for i in range(3)])
for i in range(80):
    for j in range(80):
        phases[0,i,j] = 1 - phases[1,i,j] - phases[2,i,j] 
spctrs = np.array([np.loadtxt("Data/aspim020_3ph_nps_gauss_noisespectrum_p" + str(i) + ".asc" ) for i in range(3)])
spctrs = [i/np.sum(i)*200 for i in spctrs]

# load the scale of the spectra (used to plot spectra)
x_scale = Gaussians().x

p_matr = []
a_matr = []
d_matr = []

### The parameters
* NUM_PHASES: the number of phases in the data
* l: the number of execution of the algorithm to make
* MUs: Each 'MU' corredponds to a regularization. The two new one are MU_FRO (frobenius norm of D) and MU_SPARS for the sparsity regularization.
* init_methods: can be either `random`, `average`, `smart` or `plan`. The firtst one is the default, the second assigns the average spectra of the data to the first column of D, the third one is the new initialization implemented during the internship, and the last one initialize randomly the spectra but in the same subspace as the Data.
* first_column_regularized: set it to `True` for the new regularization not to affect the "matrix" endemember (first column of D and A).

In [ ]:
NUM_PHASES = 3
l = 1
MU_LAPLs = [0.]
MU_ABSs = [0.]
MU_FROs = [0.]
MU_SPARSs = [0.1]
first_column_regularizeds = [True]
init_methods = ["average"]

### The algorithm

In [ ]:
for i in range(l):
    print("i = " + str(i))
    MU_LAPL, MU_ABS, MU_FRO, MU_SPARS = MU_LAPLs[i], MU_ABSs[i], MU_FROs[i], MU_SPARSs[i]
    init_method,  first_column_regularized = init_methods[i], first_column_regularizeds[i]
    
    dl = DictionaryLearningAlgorithm(num_phases=NUM_PHASES, mu_lapl=MU_LAPL, mu_abs=MU_ABS, mu_fro=MU_FRO, mu_spars=MU_SPARS)
    p_mat, a_mat = dl.fit(X, max_iter=2000, tol_function=0.001, initialize_method=init_method, matrix_regularized=first_column_regularized)
    p_matr.append(p_mat)
    a_matr.append(a_mat)
    d_matr.append(dl.d_matr)

### Permutations

Not necessary. It is used to print the next graphics with each phases in the right order

In [ ]:
p1 = np.argmin([MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,2])])
p2 = np.argmin([MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,2])]) 
p3 = np.argmin([MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,2])])

permutation = [p1,p2,p3]

p_matr = np.array(p_matr)
d_matr = np.array(d_matr)
a_matr = np.array(a_matr)

temp_d = d_matr[i]
temp_p = p_matr[i]
temp_a = a_matr[i]

d_matr[i] = temp_d[:,permutation]
a_matr[i] = temp_a[permutation,:]
p_matr[i] = temp_p[:,permutation]

### The spectra
Prints the spectra

In [ ]:
d_mat = d_matr[i]

fig1 = plt.figure(figsize=(20, 5))
plt.subplot(131)
plt.plot(x_scale, spctrs[0], label='real')
plt.plot(x_scale, d_mat[: , 0], label='learned')
plt.legend(loc="upper right")
plt.xlim(0, 8)
plt.xlabel("Energy [kEv]")
plt.ylabel("Intensity")
plt.title("Learned spectrum of phase 1")

plt.subplot(132)
plt.plot(x_scale, spctrs[1], label='real')
plt.plot(x_scale, d_mat[: , 1], label='learned')
plt.legend(loc="upper right")
plt.xlim(0, 8)
plt.xlabel("Energy [kEv]")
plt.ylabel("Intensity")
plt.title("Learned spectrum of phase 2")

plt.subplot(133)
plt.plot(x_scale, spctrs[2], label='real' )
plt.plot(x_scale, d_mat[: , 2], label='learned')
plt.legend(loc="upper right")
plt.xlim(0, 8)
plt.xlabel("Energy [kEv]")
plt.ylabel("Intensity")
plt.title("Learned spectrum of phase 3")
plt.show()

### Activation map

prints the activation maps

In [ ]:
fig2 = plt.figure(figsize=(16.5, 4.5))
plt.subplot(131)
plt.imshow(a_matr[i].T.reshape(*X.shape[:2], NUM_PHASES)[:, :, 0], cmap="viridis")
#plt.imshow(phases.T.reshape(*dl.x_shape[:2], NUM_PHASES)[:, :, 0].T, cmap="viridis")
plt.grid(b=30)
plt.title("Activations of first spectrum")
plt.colorbar()
plt.clim(0, 1)

plt.subplot(132)
plt.imshow(a_matr[i].T.reshape(*X.shape[:2], NUM_PHASES)[:, :, 1], cmap="viridis")
#plt.imshow(phases.T.reshape(*dl.x_shape[:2], NUM_PHASES)[:, :, 1].T, cmap="viridis")
plt.grid(b=30)
plt.title("Activations of second spectrum")
plt.colorbar()
plt.clim(0, 1)

plt.subplot(133)
plt.imshow(a_matr[i].T.reshape(*X.shape[:2], NUM_PHASES)[:, :, 2], cmap="viridis")
#plt.imshow(phases.T.reshape(*dl.x_shape[:2], NUM_PHASES)[:, :, 2].T, cmap="viridis")
plt.grid(b=30)
plt.title("Activations of third spectrum")
plt.colorbar()
plt.clim(0, 1)

### Planar projection

prints the planar projection of the data, with comparaison with the real spectra.

In [ ]:
Y = X.reshape((X.shape[0] * X.shape[1], X.shape[2]))  # flatten X

pca = PCA(n_components=2)
pca.fit(Y)
Y = pca.transform(Y)

fig = plt.figure()
color=['green','red','cyan','magenta','yellow']
#label=['µ = ' + str(MU_DCs[i]) for i in range(l)]
plt.scatter(Y[:,0],Y[:,1], color='blue')

rs = pca.transform(spctrs)
plt.scatter(rs[:,0],rs[:,1], color='black', label = "real")

for i in range(l):
    s1 = pca.transform([d_matr[i][:,0]])
    s2 = pca.transform([d_matr[i][:,1]])
    s3 = pca.transform([d_matr[i][:,2]])
    ss = np.array([s1,s2,s3,s1])

    #plt.scatter(ss[:,0,0],ss[:,0,1], label=label[i], color=color[i])
    plt.scatter(ss[:,0,0],ss[:,0,1], color=color[i])
    
#fig.savefig(name)
plt.legend(loc='upper right', numpoints=1, ncol=2, fontsize=8)
plt.show()

### Metrics

Give the result of the Spectral Angle and MSE

In [ ]:
angle_res = []
mse_res   = []

for i in range(l):
    angle_res.append([min(MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[0], d_matr[i][:,2])),   \
                      min(MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[1], d_matr[i][:,2])),   \
                      min(MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,0]), MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,1]), MetricsUtils.spectral_angle(spctrs[2], d_matr[i][:,2]))])       
    mse_res.append([MetricsUtils.MSE_map(phases[0],a_matr[i][0].reshape(*dl.x_shape[:2])),                \
                     MetricsUtils.MSE_map(phases[1],a_matr[i][1].reshape(*dl.x_shape[:2])),                 \
                     MetricsUtils.MSE_map(phases[2],a_matr[i][2].reshape(*dl.x_shape[:2])) ])
angle_res = np.array(angle_res)       
mse_res = np.array(mse_res)

print(angle_res)
print(mse_res)